In [ ]:
# 필요한 모듈 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# 한글 처리
plt.rcParams['font.family'] ='NanumGothic'
plt.rcParams['axes.unicode_minus'] =False

In [ ]:
# test 데이터 확인하기
test = pd.read_csv('test.csv', encoding='utf-8')
test['좌표X'].isnull().sum()

In [ ]:
test.info()

In [ ]:
test.isnull().sum()

In [ ]:
# 학습 데이터 로드
df = pd.read_csv('train.csv', encoding='utf-8')
df.head()

In [ ]:
df.info()

### 데이터 체크 및 Feature Engineering 고찰

In [ ]:
# 연도별 데이터 분포
df['계약년'] = df['계약년월'].astype('str').str[:4]
df['계약년'].value_counts().plot(kind='bar')
plt.title('연도별 데이터 분포')
plt.show()
# plt.savefig('./visualization/연도별 데이터 분포.png')

# 일자 feature 생성
df['일자'] = df['계약년월'].astype('str') + df['계약일'].astype('str').str.zfill(2)

# 결측치 확인
null_per = df.isnull().mean()*100
null_per.sort_values(ascending=False)

df.isnull().sum().plot(kind='bar', figsize=(20, 6))

# 결측치 70%이상 열 제거
not_null_cols = df.columns[(df.isnull().mean()*100) < 70 ] # 878000

df_origin = df.copy()
df_ex1 = df[not_null_cols]

# 번지, 본번, 부번 열 제거
# cond = df_ex1.columns.isin(['본번', '부번']) # '번지', 
# df_ex2 = df_ex1.loc[:, ~cond]
df_ex2 = df_ex1.copy()

# 등기신청일자, 거래유형, 중개사소재지
# 등기신청일자 1103907 빈값(null값으로 측정안된 데이터 다수 빼기)
# 거래유형 1086451 '-'(결측치 많음 빼기)
# 중개사소재지 1089581 '-'(중요성 떨어져서 빼기)
print((df_ex2['등기신청일자'] == ' ').sum())
print((df_ex2['거래유형'] == '-').sum())
print((df_ex2['중개사소재지'] == '-').sum())

df_ex2['거래유형'].unique()

use_cols = ['시군구', '번지', '본번', '부번', '아파트명', '전용면적(㎡)', '계약년월', '계약일', '층', '건축년도', '도로명', 'target']
df_train = df_ex2[use_cols]

# 아파트명 전처리
# 아파트명 결측치 처리 : 1118822 중에 2126 0.0019 수준으로 미미하기에 nan값 제거
cond = df_train['아파트명'].isnull() # 2126
df_train = df_train[~cond].reset_index(drop=True)
df_train.info()

df_train.shape

# 결측치 처리 완료

# 이상치 처리
df_train['층'].unique()

df_train[df_train['층'] == 1] # 66777
df_train[df_train['층'] == 2] # 76638
df_train[df_train['층'] == 3] # 80174
df_train[df_train['층'] == 4] # 81639

# 개수 확인
print((df_train['층'] == -1).sum())
print((df_train['층'] == -2).sum())
print((df_train['층'] == -3).sum())
print((df_train['층'] == -4).sum())

# 마이너스 층은 이상치로 제거
cond = df_train['층'] < 0
df_train = df_train[~cond].reset_index(drop=True)

# 층 수가 전체 층을 의미하지 않으며 각 아파트 최고층이 모두 다 다르기 때문에 비교에 의미가 없다고 판단
# 층 수 5층 이하/15층 이하/15층 초과
df_train.groupby('층')['시군구'].count().plot(kind='bar', figsize=(15, 6))

# 계약일은 의미 없다고 판단
df_train = df_train.drop('계약일', axis=1)

# 시군구 데이터로 구 열 생성
df_train['구'] = df_train['시군구'].str.split(' ').str[1]

# 202001 데이터부터 사용(전략1)
train = df_train[df_train['계약년월'] >= 202001].reset_index(drop=True)

# 201501 데이터부터 사용(과적합, 데이터 분포보니 2015가 가장 많았음)(전략2)
train = df_train[df_train['계약년월'] >= 201501].reset_index(drop=True)

# 계약년월 str로 변경
train['계약년월'] = train['계약년월'].astype('str')

# test 데이터 처리
use_cols = ['시군구', '번지', '본번', '부번', '아파트명', '전용면적(㎡)', '계약년월', '층', '건축년도', '도로명']
test = test[use_cols]

test['구'] = test['시군구'].str.split(' ').str[1]

test[test['아파트명'].isnull()] # 직접 찾아서 넣기

test['계약년월'] = test['계약년월'].astype('str')

# 경제지표 추가
int_data = pd.read_csv('./ml-outdata/ECOS 한국은행 경제통계시스템 회사채금리(2007.1-2025.8).csv', encoding='utf-8')
int_data = int_data.iloc[0, 4:].reset_index().set_axis(['계약년월', '회사채금리'], axis=1)
int_data['계약년월'] = int_data['계약년월'].str.replace('/', '')

apt_idx = pd.read_csv('./ml-outdata/KOSIS_서울시 아파트 매매가격지수_2007-2023.csv', encoding='cp949')
apt_idx = apt_idx.iloc[0, 2:].reset_index().set_axis(['계약년월', '매매가격지수'], axis=1)
apt_idx['계약년월'] = apt_idx['계약년월'].str.replace('.', '')

con_idx = pd.read_csv('./ml-outdata/KOSIS 국가통계포털 - 건설공사비지수(2020.01~2025.06) utf-8.csv', encoding='utf-8')
con_idx = con_idx.iloc[0, 3:].reset_index().set_axis(['계약년월', '건설공사비지수'], axis=1)
con_idx['계약년월'] = con_idx['계약년월'].str.replace('.', '')


apt_mount = pd.read_csv('./ml-outdata/아파트 거래량.csv', encoding='utf-8')
apt_mount['년/월'] = apt_mount['년/월'].astype('str')
apt_mount = apt_mount[['년/월', '매매']].set_axis(['계약년월', '거래량'], axis=1)


train = train.merge(int_data, how='left', on='계약년월')
train = train.merge(apt_idx, how='left', on='계약년월')
train = train.merge(con_idx, how='left', on='계약년월')
train = train.merge(apt_mount, how='left', on='계약년월')


# test 데이터도 동일하게 데이터 추가
test = test.merge(int_data, how='left', on='계약년월')
test = test.merge(apt_idx, how='left', on='계약년월')
test = test.merge(con_idx, how='left', on='계약년월')
test = test.merge(apt_mount, how='left', on='계약년월')

# 동 확인
train['동'] = train['시군구'].str.split(' ').str[-1]
train[train['동'].str.contains('천호동')]

# 동별 버스정류장 수 데이터 추가
bus = pd.read_csv('./ml-outdata/bus_feature_moongs.csv', encoding='utf-8')
bus['시군구'] = bus['si'] + ' ' + bus['gu'] + ' ' + bus['dong']
bus_count = bus.groupby('시군구')['정류소번호'].agg(버스정류장수='count').reset_index()

train = train.merge(bus_count, how='left', on='시군구')
train['버스정류장수'].isnull().sum()

# nan값은 버스정류장이 없다는 뜻이므로 0으로 처리
train['버스정류장수'] = train['버스정류장수'].fillna(0)

# 동별 지하철 수 데이터 추가
subway = pd.read_csv('./ml-outdata/subway_feature_raddr_imeanseo.csv')
subway_count = subway.groupby('full_address')['역사명'].agg(지하철수='count').reset_index()
subway_count = subway_count.rename(columns={'full_address':'시군구'})

train = train.merge(subway_count, how='left', on='시군구')
train['지하철수'].isnull().sum()

# nan값은 지하철이 없다는 뜻이므로 0으로 처리
train['지하철수'] = train['지하철수'].fillna(0)

# test 데이터도 동일하게 데이터 추가
test = test.merge(bus_count, how='left', on='시군구')
test = test.merge(subway_count, how='left', on='시군구')

# test 데이터 버스정류장 수, 지하철 수도 동일하게 null값은 0으로 처리
test['버스정류장수'] = test['버스정류장수'].fillna(0)
test['지하철수'] = test['지하철수'].fillna(0)

# 연식(계약년-건축년도) featrue 생성
# train
train['계약년'] = train['계약년월'].str[:4]
train['계약월'] = train['계약년월'].str[4:]

train['계약년'] = train['계약년'].astype('int')
train['연식'] = train['계약년'] - train['건축년도']

# test
test['계약년'] = test['계약년월'].str[:4]
test['계약월'] = test['계약년월'].str[4:]

test['계약년'] = test['계약년'].astype('int')
test['연식'] = test['계약년'] - test['건축년도']

fill_values = {
    2451: "740-7",
    2452: "743-28",
    2453: "747-34",
    2454: "752-17",
    2455: "780-86",
    4035: "INS새터아파트",
    4449: "상진아파트",
    8041: "유탑유블레스아파트",
    8042: "유탑유블레스아파트",
    8966: "432-904",
}

for idx, value in fill_values.items():
    test.loc[idx, "아파트명"] = value

train.loc[train['연식'] < 0, '연식'] = 0

# 번지 null 처리
train['번지'].unique()
train[train['본번'].isnull()]['아파트명'].unique()
train[train['번지'].isnull()]['아파트명'].unique()
test[test['번지'].isnull()]['아파트명'].unique()

# 번지 null처리
train.loc[train['본번'].isnull(), '번지'] = '557'
train.loc[train['번지'].isnull(), '번지'] = '384'
train = train.drop(['본번', '부번'], axis=1)

test.loc[test['번지'].isnull(), '번지'] = '384'
test = test.drop(['본번', '부번'], axis=1)

# 데이터 확인
train.info()
test.info()

# eda 데이터 생성
# train.to_csv('train_for_eda_2015.csv', encoding='utf-8', index=False)
# test.to_csv('test_for_eda_2015.csv', encoding='utf-8', index=False)